In [12]:
# Import statements

import glob
import re
import parselmouth
from parselmouth.praat import call

In [15]:
# Pitch, intensity, duration and F0 for singing features.
singing = []
counter = 0 

for wav_file in glob.glob("/Users/adrizhao/Documents/NLP/nlp-final-project/singing/*.wav"):    


# Get duration, mean pitch, mean intensity
    sound = parselmouth.Sound(wav_file)
    pitch = call(sound, "To Pitch", 0, 75, 600) 
    meanpitch = call(pitch, "Get mean", 0, 0, "Hertz")
    intensity = call(sound, "To Intensity", 75, 0, "yes")
    meanintensity = call(intensity, "Get mean", 0, 0, "energy")
    duration = call(sound, "Get total duration")

    
    # get mean F0 (vowels only)
    formant = call(sound, "To Formant (burg)", 0, 5, 5500, 0.025, 50)
    tg_file = re.sub("wav", "TextGrid", wav_file)
    textgrid = call("Read from file", tg_file)
    intv = call(textgrid, "Get number of intervals", 1)
    vowels = 0
    f_zero = 0
    for i in range(1, intv):
        phone = call(textgrid, "Get label of interval", 1, i)
        if re.match('[AEIOU]', phone.lower):
            vowels += 1
            vowel_onset = call(textgrid, "Get starting point", 1, i)
            vowel_offset = call(textgrid, "Get end point", 1, i)
            midpoint = vowel_onset + ((vowel_offset - vowel_onset) / 2)
            f_zero += call(formant, "Get value at time", 1, midpoint, "Hertz", "Linear")
            
    singing.append([meanpitch, meanintensity, duration, (f_zero/vowels)])

5


AttributeError: 'NoneType' object has no attribute 'lower'

In [8]:
#jitter and shimmer

# We'll write this info out to a file.
f = open("/Users/adrizhao/Documents/NLP/nlp-final-project/jitter_shimmer.csv", "w")

f.write("filename, jitter, shimmers, hnr, singvspeak" + "\n")

for wav_file in glob.glob("/Users/adrizhao/Documents/NLP/nlp-final-project/singing/*.wav"):
    singvspeak = 0
    filename = re.sub("^.*\/", "", wav_file)
    tg_file = re.sub("wav", "TextGrid", filename)
    textgrid = call("Read from file", "singing/" + tg_file)
    intv = call(textgrid, "Get number of intervals", 1)
    sound = parselmouth.Sound( "singing/" + filename)
    pointProcess = call(sound, "To PointProcess (periodic, cc)", 75, 600)
    harmonicity = call(sound, "To Harmonicity (cc)", 0.01, 75, 0.1, 1.0)
    jitter = 0.0
    shimmer = 0.0
    hnr = 0.0
    count = 0
    for i in range(1, intv):
        phone = call(textgrid, "Get label of interval", 1, i)
        if re.match('[AEIOU]', phone):
            vowel_onset = call(textgrid, "Get starting point", 1, i)
            vowel_offset = call(textgrid, "Get end point", 1, i)
            j = call(pointProcess, "Get jitter (local)", vowel_onset, vowel_offset, 0.0001, 0.03, 1.3)
            s = call([sound, pointProcess], "Get shimmer (local)", vowel_onset, vowel_offset, 0.0001, 0.02, 1.3, 1.6)
            h = call(harmonicity, "Get mean", vowel_onset, vowel_offset)
            if j >= 0 and s >= 0 and h >= 0:
                count += 1
                jitter += j
                shimmer += s
                hnr += h
    f.write(filename + "," + str(jitter/count) + "," + str(shimmer/count) + "," + str(hnr/count) + "," + str(singvspeak) + "\n")
    
for wav_file in glob.glob("/Users/adrizhao/Documents/NLP/nlp-final-project/speaking/*.wav"):
    singvspeak = 1
    filename = re.sub("^.*\/", "", wav_file)
    tg_file = re.sub("wav", "TextGrid", filename)
    textgrid = call("Read from file", "speaking/" + tg_file)
    intv = call(textgrid, "Get number of intervals", 1)
    sound = parselmouth.Sound( "speaking/" + filename)
    pointProcess = call(sound, "To PointProcess (periodic, cc)", 75, 600)
    harmonicity = call(sound, "To Harmonicity (cc)", 0.01, 75, 0.1, 1.0)
    jitter = 0.0
    shimmer = 0.0
    hnr = 0.0
    count = 0
    for i in range(1, intv):
        phone = call(textgrid, "Get label of interval", 1, i)
        if re.match('[AEIOU]', phone):
            vowel_onset = call(textgrid, "Get starting point", 1, i)
            vowel_offset = call(textgrid, "Get end point", 1, i)
            j = call(pointProcess, "Get jitter (local)", vowel_onset, vowel_offset, 0.0001, 0.03, 1.3)
            s = call([sound, pointProcess], "Get shimmer (local)", vowel_onset, vowel_offset, 0.0001, 0.02, 1.3, 1.6)
            h = call(harmonicity, "Get mean", vowel_onset, vowel_offset)
            if j >= 0 and s >= 0 and h >= 0:
                count += 1
                jitter += j
                shimmer += s
                hnr += h
    f.write(filename + "," + str(jitter/count) + "," + str(shimmer/count) + "," + str(hnr/count) + "," + str(singvspeak) + "\n")
f.close()

None


TypeError: expected string or bytes-like object